In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import csv

In [ ]:
train_dir = "../input/train.csv"
test_dir = "../input/test.csv"
n_classes = 10

In [ ]:
df = pd.read_csv(train_dir)

In [ ]:
temp_labels = df.loc[:,'label']
temp_train = df.loc[:,df.columns!="label"]

In [ ]:
labels = []
for i in range(len(temp_labels)):
    labels.append(temp_labels[i])

In [ ]:
data = temp_train.values.tolist()
data = np.array(data)
data = data.astype('float32')/255.0

In [ ]:
labels = np.array(labels)
labels = keras.utils.to_categorical(labels)

In [ ]:
print(data.shape)
print(labels.shape)

In [ ]:
model = Sequential()
model.add(Dense(784, activation = 'relu', input_shape = (784,)))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(264, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))
print("GENERAL MODEL :-")
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adadelta(), metrics = ['accuracy'])

In [ ]:
model.fit(data, labels, batch_size = 32, epochs = 9, validation_split = 0.1)

In [ ]:
def get_prediction_classes(prediction_array):
    predictions = []
    for x in prediction_array:
        pos = None
        current = 0
        for i in range(len(x)):
            if x[i] > current:
                current = x[i]
                pos = i
        predictions.append(pos)
    return predictions

In [ ]:
def make_prediction_general_model(test):
    test = np.array(test)
    test = test.astype('float32')/255.0
    prediction_array = model.predict_on_batch(test)
    predictions = get_prediction_classes(prediction_array)
    return predictions

In [ ]:
test_data = pd.read_csv(test_dir)
test_data = test_data.values.tolist()

**USING CNN MODEL **:

In [ ]:
X_train_images = temp_train.values.tolist()
X_train_images = np.array(X_train_images)
print(X_train_images.shape)
X_train_shape = X_train_images.reshape(42000, 28 ,28, 1)

In [ ]:
X_train_final = X_train_shape.astype('float32')/255.0

In [ ]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size = [3,3], activation = 'relu', input_shape = (28,28,1)))
cnn_model.add(Conv2D(64, kernel_size = [3,3], activation = 'relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPool2D(pool_size = [2,2], strides = 2))
cnn_model.add(Conv2D(128, kernel_size = [3,3], activation = 'relu'))
cnn_model.add(MaxPool2D(pool_size = [2,2], strides = 2))
cnn_model.add(Flatten())
cnn_model.add(Dense(512, activation = 'relu'))
cnn_model.add(Dense(10, activation = 'softmax'))
print("CONVOLUTIONAL NEURAL NETWORK MODEL :-")
cnn_model.summary()

In [ ]:
aug_cnn_model = cnn_model

In [ ]:
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
cnn_model.fit(X_train_final, labels, batch_size = 32, epochs = 6, validation_split = 0.1)

In [ ]:
def make_predictions_CNN_model(test):
    test = np.array(test)
    test_reshaped = test.reshape(test.shape[0], 28, 28, 1)
    test_reshaped = test_reshaped.astype('float')/255.0
    prediction_array = cnn_model.predict_on_batch(test_reshaped)
    predictions = get_prediction_classes(prediction_array)
    return predictions

**ADDING DATA AUGMENTATION :**

In [ ]:
X_train_aug, X_test_aug, Y_train_aug, Y_test_aug = train_test_split(X_train_final, labels, test_size = 0.1)

In [ ]:
data_aug = ImageDataGenerator(featurewise_center = False,
                             samplewise_center = False,
                             featurewise_std_normalization = False,
                             samplewise_std_normalization = False,
                             zca_whitening = False,
                             rotation_range = 15,
                             zoom_range = 0.1,
                             width_shift_range = 0.1,
                             height_shift_range = 0.1,
                             horizontal_flip = False,
                             vertical_flip = False)

In [ ]:
data_aug.fit(X_train_aug)

In [ ]:
aug_cnn_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
print("CONVOLUTIONAL NEURAL NETWORK MODEL WITH DATA AUGMENTATION :-")
aug_model_history = aug_cnn_model.fit_generator(data_aug.flow(X_train_aug, Y_train_aug, batch_size = 32),
                                               epochs = 1, validation_data = (X_test_aug, Y_test_aug),
                                               steps_per_epoch = X_train_aug.shape[0])

In [ ]:
def make_predictions_CNN_aug_model(test):
    test = np.array(test)
    test_reshaped = test.reshape(test.shape[0], 28, 28, 1)
    test_reshaped = test_reshaped.astype('float')/255.0
    prediction_array = aug_cnn_model.predict_on_batch(test_reshaped)
    predictions = get_prediction_classes(prediction_array)
    return predictions

In [ ]:
predictions_for_aug = make_predictions_CNN_aug_model(test_data)
print(len(predictions_for_aug))
print(predictions_for_aug[:10])

In [ ]:
final_csv = []
csv_title = ['ImageId', 'Label']
final_csv.append(csv_title)
for i in range(len(predictions_for_aug)):
    image_id = i + 1
    label = predictions_for_aug[i]
    temp = [image_id, label]
    final_csv.append(temp)

print(len(final_csv))

with open('submission_csv_aug.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(final_csv)
file.close()
